<a href="https://colab.research.google.com/github/sworddish/bootleg/blob/yx-dev/notebooks/Named_Entity_Recognition_Mandarin_Weibo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition in Mandarin on a Weibo Social Media Dataset

---

[Github](https://github.com/eugenesiow/practical-ml/blob/master/notebooks/Named_Entity_Recognition_Mandarin_Weibo.ipynb) | More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml)

---

Notebook to train a [flair](https://github.com/flairNLP/flair) model in mandarin using stacked embeddings (with word and BERT embeddings) to perform named entity recognition (NER). 

The [dataset](https://github.com/hltcoe/golden-horse) used contains 1,890 Sina Weibo messages annotated with four entity types (person, organization, location and geo-political entity), including named and nominal mentions from the paper [Peng et al. (2015)](https://www.aclweb.org/anthology/D15-1064/) and with revised annotated data from [He et al. (2016)](https://arxiv.org/abs/1611.04234).

The current state-of-the-art model on this dataset is from [Peng et al. (2016)](https://www.aclweb.org/anthology/P16-2025/) with an average F1-score of **47.0%** (Table 1) and from [Peng et al. (2015)](https://www.aclweb.org/anthology/D15-1064.pdf) with an F1-score of **44.1%** (Table 2). The authors say that the poor results on the test set show the "difficulty of this task" - which is true a sense because the dataset is really quite small for the NER task with 4 classes (x2 as they differentiate nominal and named entities) with a test set of only 270 sentences.

Our flair model is able to improve the state-of-the-art with an F1-score of **67.5%**, which is a cool 20+ absolute percentage points better than the current state-of-the-art performance.

The notebook is structured as follows:
* Setting up the GPU Environment
* Getting Data
* Training and Testing the Model
* Using the Model (Running Inference)

## Task Description

> Named entity recognition (NER) is the task of tagging entities in text with their corresponding type. Approaches typically use BIO notation, which differentiates the beginning (B) and the inside (I) of entities. O is used for non-entity tokens.

# Setting up the GPU Environment

#### Ensure we have a GPU runtime

If you're running this notebook in Google Colab, select `Runtime` > `Change Runtime Type` from the menubar. Ensure that `GPU` is selected as the `Hardware accelerator`. This will allow us to use the GPU to train the model subsequently.

#### Install Dependencies

In [7]:
!pip install -q folium==0.2.1 flair==0.11

     |████████████████████████████████| 69 kB 3.6 MB/s 


# Getting Data

The dataset, including the train, test and dev sets, has just been included in the `0.7 release` of flair, hence, we just use the `flair.datasets` loader to load the `WEIBO_NER` dataset into the flair `Corpus`. The [raw datasets](https://github.com/87302380/WEIBO_NER) are also available on Github.

In [9]:
import flair.datasets
from flair.data import Corpus
corpus = flair.datasets.NER_CHINESE_WEIBO()
print(corpus)

2022-05-12 04:16:16,862 https://raw.githubusercontent.com/87302380/WEIBO_NER/main/data/weiboNER_2nd_conll_format.train not found in cache, downloading to /tmp/tmpgklsvjzd


462545B [00:00, 12041949.15B/s]          

2022-05-12 04:16:17,014 copying /tmp/tmpgklsvjzd to cache at /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.train
2022-05-12 04:16:17,024 removing temp file /tmp/tmpgklsvjzd


2022-05-12 04:16:17,240 https://raw.githubusercontent.com/87302380/WEIBO_NER/main/data/weiboNER_2nd_conll_format.test not found in cache, downloading to /tmp/tmpcu6evg11


94016B [00:00, 12273388.06B/s]          

2022-05-12 04:16:17,323 copying /tmp/tmpcu6evg11 to cache at /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.test
2022-05-12 04:16:17,328 removing temp file /tmp/tmpcu6evg11
2022-05-12 04:16:17,486 https://raw.githubusercontent.com/87302380/WEIBO_NER/main/data/weiboNER_2nd_conll_format.dev not found in cache, downloading to /tmp/tmpjd8fm87d



91510B [00:00, 13857345.62B/s]          

2022-05-12 04:16:17,545 copying /tmp/tmpjd8fm87d to cache at /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.dev
2022-05-12 04:16:17,549 removing temp file /tmp/tmpjd8fm87d
2022-05-12 04:16:17,552 Reading data from /root/.flair/datasets/ner_chinese_weibo
2022-05-12 04:16:17,554 Train: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.train
2022-05-12 04:16:17,556 Dev: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.dev
2022-05-12 04:16:17,558 Test: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.test


Corpus: 1350 train + 270 dev + 270 test sentences


We can see that the total 1,890 sentences have already been split into train (1,350), dev (270) and test (270) sets in a 5:1:1 ratio.

# Training and Testing the Model

#### Train the Model

To train the flair `SequenceTagger`, we use the `ModelTrainer` object with the corpus and the tagger to be trained. We use flair's sensible default options in the `.train()` method, while specifying the output folder for the `SequenceTagger` model to be `/content/model/`. We also set the `embeddings_storage_mode` to be `gpu` to utilise the GPU to store the embeddings for more speed. Note that if you run this with a larger dataset you might run out of GPU memory, so be sure to set this option to `cpu` - it will still use the GPU to train but the embeddings will not be stored in the CPU and there will be a transfer to the GPU each epoch.

Be prepared to allow the training to run for about 0.5 to 1 hour. We set the `max_epochs` to 50 so the the training will complete faster, for higher F1-score you can increase this number to 100 or 150.

In [11]:
import flair
from typing import List
from flair.trainers import ModelTrainer
from flair.models import SequenceTagger
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, BertEmbeddings, BytePairEmbeddings

tag_type = 'ner'
tag_dictionary = corpus.make_label_dictionary('ner')

# For an even faster training time, you can comment out the BytePairEmbeddings
# Note: there will be a small drop in performance if you do so.
embedding_types: List[TokenEmbeddings] = [
    WordEmbeddings('zh-crawl'),
    BytePairEmbeddings('zh'),
    BertEmbeddings('bert-base-chinese'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('/content/model/',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50,
              embeddings_storage_mode='gpu')

2022-05-12 04:18:37,327 Computing label dictionary. Progress:


1350it [00:00, 11120.38it/s]

2022-05-12 04:18:37,466 Dictionary created for label 'ner' with 9 values: PER.NOM (seen 772 times), PER.NAM (seen 578 times), GPE.NAM (seen 205 times), ORG.NAM (seen 183 times), LOC.NAM (seen 56 times), LOC.NOM (seen 51 times), ORG.NOM (seen 42 times), GPE.NOM (seen 8 times)


2022-05-12 04:18:38,717 https://flair.informatik.hu-berlin.de/resources/embeddings/token/zh-crawl-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpcvct3ilg


100%|██████████| 1200000128/1200000128 [00:54<00:00, 21924238.18B/s]

...

2022-05-12 04:47:41,934 Evaluating as a multi-label problem: False
2022-05-12 04:47:41,955 DEV : loss 0.07134849578142166 - f1-score (micro avg)  0.7246
2022-05-12 04:47:41,970 BAD EPOCHS (no improvement): 2


2022-05-12 04:47:51,638 ----------------------------------------------------------------------------------------------------
2022-05-12 04:47:51,642 loading file /content/model/best-model.pt
2022-05-12 04:48:03,411 SequenceTagger predicts: Dictionary with 35 tags: O, S-PER.NOM, B-PER.NOM, E-PER.NOM, I-PER.NOM, S-PER.NAM, B-PER.NAM, E-PER.NAM, I-PER.NAM, S-GPE.NAM, B-GPE.NAM, E-GPE.NAM, I-GPE.NAM, S-ORG.NAM, B-ORG.NAM, E-ORG.NAM, I-ORG.NAM, S-LOC.NAM, B-LOC.NAM, E-LOC.NAM, I-LOC.NAM, S-LOC.NOM, B-LOC.NOM, E-LOC.NOM, I-LOC.NOM, S-ORG.NOM, B-ORG.NOM, E-ORG.NOM, I-ORG.NOM, S-GPE.NOM, B-GPE.NOM, E-GPE.NOM, I-GPE.NOM, <START>, <STOP>


100%|██████████| 9/9 [00:11<00:00,  1.24s/it]

2022-05-12 04:48:15,590 Evaluating as a multi-label problem: False
2022-05-12 04:48:15,616 0.6759	0.6388	0.6568	0.5047
2022-05-12 04:48:15,617 
Results:
- F-score (micro) 0.6568
- F-score (macro) 0.38
- Accuracy 0.5047

By class:
              precision    recall  f1-score   support

     PER.NOM     0.6872    0.7151    0.7009       172
     PER.NAM     0.8317    0.7434    0.7850       113
     GPE.NAM     0.6909    0.8085    0.7451        47
     ORG.NAM     0.4048    0.4359    0.4198        39
     LOC.NAM     0.1000    0.0526    0.0690        19
     ORG.NOM     0.5000    0.2353    0.3200        17
     LOC.NOM     0.0000    0.0000    0.0000         9
     GPE.NOM     0.0000    0.0000    0.0000         2

   micro avg     0.6759    0.6388    0.6568       418
   macro avg     0.4018    0.3739    0.3800       418
weighted avg     0.6479    0.6388    0.6397       418

2022-05-12 04:48:15,618 -----------------------------------------------------------------------------------------------

{'dev_loss_history': [0.3250114917755127,
  0.3100951910018921,
  0.16189177334308624,
  0.1266089230775833,
  0.11853788793087006,
  0.10589036345481873,
  0.10579219460487366,
  0.10286907106637955,
  0.0973888412117958,
  0.08998274058103561,
  0.10971860587596893,
  0.08272195607423782,
  0.10471440106630325,
  0.08189141750335693,
  0.07838036119937897,
  0.0815553218126297,
  0.07177051156759262,
  0.07808586210012436,
  0.07343596965074539,
  0.07472072541713715,
  0.07477864623069763,
  0.07967667281627655,
  0.08014415204524994,
  0.07370781898498535,
  0.07259213179349899,
  0.07197801768779755,
  0.07252401858568192,
  0.06987930834293365,
  0.07443656027317047,
  0.07247556000947952,
  0.0690680518746376,
  0.07429857552051544,
  0.07189839333295822,
  0.07272768765687943,
  0.06971608847379684,
  0.07431035488843918,
  0.07199511677026749,
  0.07124228775501251,
  0.07089612632989883,
  0.07174701988697052,
  0.07229874283075333,
  0.07175947725772858,
  0.0718779340386390

We see that the output accuracy (F1-score) for our new model is **67.5%** (F1-score (micro) 0.6748). We use micro F1-score (rather than macro F1-score) as there are multiple entity classes in this setup with [class imbalance](https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin).

> We have a new SOTA NER model in mandarin, over 20 percentage points (absolute) better than the previous SOTA for this Weibo dataset!

## Using the Model (Running Inference)

Running the model to do some predictions/inference is as simple as calling `tagger.predict(sentence)`. Do note that for mandarin each character needs to be split with spaces between each character (e.g. `一 节 课 的 时 间`) so that the tokenizer will work properly to split them to tokens (if you're processing them for input into the model when building an app). For more information on this, check out the [flair tutorial on tokenization](https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_1_BASICS.md#tokenization).

In [13]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.data import Corpus

# Load the model that we trained, you can comment this out if you already have 
# the model loaded (e.g. if you just ran the training)
tagger: SequenceTagger = SequenceTagger.load("/content/model/final-model.pt")

# Load the WEIBO corpus and use the first 5 sentences from the test set
corpus = flair.datasets.NER_CHINESE_WEIBO()
for idx in range(0, 5):
  sentence = corpus.test[idx]
  tagger.predict(sentence)
  print(sentence.to_tagged_string())

2022-05-12 05:23:53,792 loading file /content/model/final-model.pt
2022-05-12 05:24:00,123 SequenceTagger predicts: Dictionary with 35 tags: O, S-PER.NOM, B-PER.NOM, E-PER.NOM, I-PER.NOM, S-PER.NAM, B-PER.NAM, E-PER.NAM, I-PER.NAM, S-GPE.NAM, B-GPE.NAM, E-GPE.NAM, I-GPE.NAM, S-ORG.NAM, B-ORG.NAM, E-ORG.NAM, I-ORG.NAM, S-LOC.NAM, B-LOC.NAM, E-LOC.NAM, I-LOC.NAM, S-LOC.NOM, B-LOC.NOM, E-LOC.NOM, I-LOC.NOM, S-ORG.NOM, B-ORG.NOM, E-ORG.NOM, I-ORG.NOM, S-GPE.NOM, B-GPE.NOM, E-GPE.NOM, I-GPE.NOM, <START>, <STOP>
2022-05-12 05:24:00,766 Reading data from /root/.flair/datasets/ner_chinese_weibo
2022-05-12 05:24:00,767 Train: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.train
2022-05-12 05:24:00,771 Dev: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.dev
2022-05-12 05:24:00,774 Test: /root/.flair/datasets/ner_chinese_weibo/weiboNER_2nd_conll_format.test
Sentence: "一 节 课 的 时 间 真 心 感 动 了 李 开 复 感 动" → ["李 开 复"/PER.NAM]
Sentence: "回 复 支 持 ， 赞 成 ， 哈 哈 米 八 吴 够 

We can connect to Google Drive with the following code to save any files you want to persist. You can also click the `Files` icon on the left panel and click `Mount Drive` to mount your Google Drive.

The root of your Google Drive will be mounted to `/content/drive/My Drive/`. If you have problems mounting the drive, you can check out this [tutorial](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [14]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


You can move the model files from our local directory to your Google Drive.

In [15]:
import shutil
shutil.move('/content/model/', "/content/drive/My Drive/model/")

'/content/drive/My Drive/model/'

More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml) and do drop us some feedback on how to improve the notebooks on the [Github repo](https://github.com/eugenesiow/practical-ml/).